# Exercise 1: Agent Hello World - Solution

Complete implementation of an AI agent using the OpenAI Agents SDK with a custom weather tool that calls the National Weather Service API.

## Setup

Import the OpenAI Agents SDK and required libraries.

In [ ]:
%pip install --quiet openai-agents httpx python-dotenv nest-asyncio

import os
import asyncio
import httpx
from dotenv import load_dotenv
from agents import Agent, Runner, function_tool

# Load environment variables only once
if not globals().get("ENV_LOADED", False):
    load_dotenv()
    ENV_LOADED = True
    env_status = "Loaded environment variables from .env"
else:
    env_status = "Environment variables already loaded"

print("\n🤖 Exercise 1: Agent Hello World - Solution\n")
print(f"✅ {env_status}")

## Step 1: Define a Weather Tool with @function_tool

Create a custom tool that calls the National Weather Service API to get real weather alerts.

In [ ]:
@function_tool
async def get_weather_alerts(state: str) -> str:
    """
    Get current weather alerts for a US state from the National Weather Service.
    
    Args:
        state: Two-letter US state code (e.g., 'CA', 'NY', 'TX')
    
    Returns:
        String describing current weather alerts
    """
    print(f"🌐 Calling NWS API for state: {state}")
    
    try:
        # Call the National Weather Service API
        url = f"https://api.weather.gov/alerts/active/area/{state.upper()}"
        headers = {"User-Agent": "OpenAI-Agents-Workshop (educational)"}
        
        async with httpx.AsyncClient() as client:
            response = await client.get(url, headers=headers, timeout=10.0)
            response.raise_for_status()
            
            data = response.json()
            features = data.get("features", [])
            
            if not features:
                print(f"✅ No active alerts")
                return f"No active weather alerts for {state.upper()}."
            
            # Extract alert information
            alerts = []
            for feature in features[:5]:  # Limit to first 5 alerts
                properties = feature.get("properties", {})
                event = properties.get("event", "Unknown")
                headline = properties.get("headline", "No headline")
                severity = properties.get("severity", "Unknown")
                alerts.append(f"- {event} ({severity}): {headline}")
            
            result = f"Active weather alerts for {state.upper()}:\n" + "\n".join(alerts)
            print(f"✅ Found {len(features)} alert(s)")
            return result
        
    except httpx.HTTPError as e:
        error_msg = f"Failed to fetch weather alerts: {str(e)}"
        print(f"❌ {error_msg}")
        return error_msg
    except Exception as e:
        error_msg = f"Unexpected error: {str(e)}"
        print(f"❌ {error_msg}")
        return error_msg

print("✅ Weather tool defined with @function_tool")
print("ℹ️  This tool makes real API calls to weather.gov")

## Step 2: Create Agent with the Weather Tool

Define the agent using the OpenAI Agents SDK with the custom weather tool.

In [ ]:
# Create agent with the custom weather tool
agent = Agent(
    name="Weather Agent",
    instructions="You help users get weather alert information for US states. Use the get_weather_alerts tool to fetch real data from the National Weather Service.",
    tools=[get_weather_alerts]
)

print("✅ Agent created with weather tool")
print("ℹ️  The agent can now fetch real weather alerts")

## Step 3: Verify OpenAI API Key

Ensure the API key is configured before running the agent.

In [ ]:
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY is not set. Add it to your .env file.")

masked_key = f"{api_key[:4]}...{api_key[-4:]}" if len(api_key) >= 8 else "***"
print(f"✅ OPENAI_API_KEY detected: {masked_key}")

✓ OPENAI_API_KEY detected: sk-s...WyoA

## Step 4: Run the Agent

Execute the agent with a weather query.

In [ ]:
async def run_solution():
    query = "Are there any weather alerts for California?"
    print(f"\nQuery: {query}\n")
    
    # Run the agent using the Agents SDK Runner
    result = await Runner.run(agent, query)
    
    # Display the final output
    print(f"\n🤖 Agent Response:\n{result.final_output}\n")
    print("ℹ️  This used a real API call to the National Weather Service!\n")

# Jupyter-specific async handling (notebooks already have an event loop)
# In a regular Python script, you would use: asyncio.run(run_solution())
try:
    loop = asyncio.get_running_loop()
    import nest_asyncio
    nest_asyncio.apply()
    await run_solution()
except RuntimeError:
    asyncio.run(run_solution())